In [7]:
from math import log
from rocketcea.cea_obj_w_units import CEA_Obj
from proptools import nozzle  


In [8]:
C = CEA_Obj(oxName="LOX", fuelName="RP-1")

Wpayload = 8500
Wstg = 5106
Wpropellant = 45920

Winit = Wstg + Wpropellant + Wpayload
Wfinal = Winit - Wpropellant

def show_deltaV( Pc=475.0, eps=84.0, MR=5.88 ):
    IspVac = C.get_Isp(Pc=Pc, MR=MR, eps=eps)
    IspDel = 0.969 * IspVac
    deltaV = 32.174 * IspDel * log( Winit / Wfinal ) # ft/sec
    print( '%8.1f %8.1f    %8.2f    %8.1f     %8.1f        %8.1f     '%(Pc, eps, MR, IspVac, IspDel, deltaV))

print(' Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)')
show_deltaV(Pc=475, eps=120)


 Pc(psia) AreaRatio  MixtureRatio   IspVac(sec)  IspDel(sec)   deltaV(ft/sec)
   475.0    120.0        5.88       322.1        312.2         14822.7     


# Missão

In [9]:
deltav_mission = 12000
payload_weight = 5000
payload_height = 6.7
payload_diameter = 4.6

# Parâmetros Fixos

In [10]:
n_stages = 2
deltav_stages = {1: 3500, 2: 8500}
stage_radii = {1: 1.8, 2: 1.8}
oxidizer = "LOX"
fuel = "RP-1"
engine_cycle = "Gas Generator"
chamber_pressure = {1: 97, 2: 97}
coef_estrutual = {1: 0.06, 2: 0.05}
max_n_engines = {1: 15, 2: 1}


# Fronteira dos parâmetros

In [11]:
mixture_ratior_border = {"begin": 1.5, "end": 3.5}
throat_diameter_border = {"begin": 0.2, "end": 0.3}

# Restrições

In [12]:
min_acceleration = {1: 1.3, 2: 0.8}
max_ld_ration = {1: 25, 2: 25}
molar_masses = {"LOX":32, "RP-1":175}

# -------

In [13]:
import math

class Engine:
    def __init__(self,
                 fuel_name:str,
                 ox_name:str,
                 mr:float,
                 pc:float,
                 nozzle_diam:float,
                 eps:float):
        """
        fuel_name: 
        ox_name (oxidizer name):
        mr (mixture ratio):
        nozzle_diam (nozzle throat diameter):
        esp (nozzle expansion ratio)
        """
        
        self.fuel_name = fuel_name
        self.ox_name = ox_name
        self.mr = mr
        self.pc = pc
        self.nozzle_diam = nozzle_diam
        self.eps = eps

        self.R_universal = 8.314462 # Constante universal dos gases

        self.C = CEA_Obj(oxName=self.ox_name, 
                         fuelName=self.fuel_name,
                         isp_units='sec',
                         cstar_units='m/s', 
                         pressure_units='MPa', 
                         temperature_units='K', 
                         sonic_velocity_units='m/s', 
                         enthalpy_units='J/g', 
                         density_units='kg/m^3', 
                         specific_heat_units='J/kg-K', 
                         viscosity_units='millipoise', 
                         thermal_cond_units='mcal/cm-K-s')


    def get_F_vac(self):
        #IspVac, Pe_vac = self.C.estimate_Ambient_Isp(Pc=self.pc, MR=self.mr, eps=self.eps, Pamb=0.00001)
        IspVac = self.C.get_Isp(Pc=self.pc, MR=self.mr, eps=self.eps)
        IspAmb, Pe_amb = self.C.estimate_Ambient_Isp(Pc=self.pc,Pamb=1e5, MR=self.mr, eps=self.eps)
        temperature_chamber, temperature_throat, temperature_exit = self.C.get_Temperatures(Pc=self.pc, MR=self.mr, eps=self.eps)
        pc_pe = self.C.get_PcOvPe(Pc=self.pc, MR=self.mr, eps=self.eps)
        chamber_velocity, throat_velocity, exit_velocity = self.C.get_SonicVelocities(Pc=self.pc, MR=self.mr, eps=self.eps)
        chamber_density, throat_density, exit_density = self.C.get_Densities(Pc=self.pc, MR=self.mr, eps=self.eps)
        exit_mach = self.C.get_MachNumber(Pc=self.pc, MR=self.mr, eps=self.eps)
        cf_vac, cf_sea, cf_mode = self.C.get_PambCf(Pc=self.pc, MR=self.mr, eps=self.eps, Pamb=1)#thrust coefficient

        nozzle_throat_area = math.pi*((self.nozzle_diam/2) **2)
        nozzle_exit_area = nozzle_throat_area * self.eps
        pe = 1/pc_pe * self.pc

        exit_speed = exit_mach * exit_velocity/1000
        mass_flow = exit_density/1000 * nozzle_exit_area * exit_speed
        thrust_vacuum = mass_flow * exit_velocity + pe  * nozzle_exit_area
        thrust_sea = mass_flow * exit_velocity + (pe-1e5)* nozzle_exit_area

        thrust_vacuum = cf_vac * self.pc * nozzle_throat_area
        thrust_sea = cf_sea * self.pc * nozzle_throat_area


        print(nozzle_exit_area)
        print("Isp Vac: " + str(IspVac),"Isp Sea: " + str(IspAmb)) 
        print("Mass flow: " + str(mass_flow))
        print(f"Thrust Vac {thrust_vacuum}", f"Thrust Sea {thrust_sea}")

        return



engine = Engine(fuel_name="RP-1", 
                ox_name="LOX",
                mr=2.3,
                pc=9.7*1e6 ,
                nozzle_diam=0.23125,
                eps=165)

engine.get_F_vac()



6.930065005430962
Isp Vac: 350.60767346744154 Isp Sea: 136.35328867508358
Mass flow: 298.4472616654054
Thrust Vac 973818.3344951972 Thrust Sea 1026147.6426383896


In [14]:
engine = Engine(fuel_name="RP-1", 
                ox_name="LOX",
                mr=2.3,
                pc=9.7*1e6 ,
                nozzle_diam=0.23125,
                eps=165)

engine.get_F_vac()


6.930065005430962
Isp Vac: 350.60767346744154 Isp Sea: 136.35328867508358
Mass flow: 298.4472616654054
Thrust Vac 973818.3344951972 Thrust Sea 1026147.6426383896


# Modelo de motor usando proptools 

In [16]:
class EngineProp:
    def __init__(self,
                 fuelName:str,
                 oxName:str,
                 MR:float,
                 Pc:float,
                 eps:float,
                 nozzleDiam = None,
                 At=None):
        """
        fuel_name: 
        ox_name (oxidizer name):
        mr (mixture ratio):
        nozzle_diam (nozzle throat diameter):
        esp (nozzle expansion ratio)
        At (Area - throat)
        """
        
        self.fuelName = fuelName
        self.oxName = oxName
        self.MR = MR
        self.Pc = Pc
        self.nozzleDiam = nozzleDiam
        self.eps = eps
        self.At = At

        # Podemos escolher informar o diametro do throat ou a area
        if ((self.At == None) and (self.nozzleDiam != None)):
            self.At = np.pi * ((self.nozzleDiam/2) ** 2)
        elif ((self.At == None) and (self.nozzleDiam == None)):
            raise("Informar At ou nozzleDiam")
        elif ((self.At != None) and (self.nozzleDiam != None)):
            raise("Informar apenas 1: At ou nozzleDiam")

        # Pressao precisa estar em MPa apesar e dados precisam ser fornecidos na ordem de 1e6
        self.ceaObj = CEA_Obj( oxName=oxName, fuelName=fuelName, pressure_units='MPa', cstar_units='m/s', temperature_units='K')

    def calcGasProperties(self):
        # mw -> Molecular weight
        # Cstar -> Characteristc velocity (Isp * g0 / Cf)
        IspVac, Cstar, Tc, mw, gamma = self.ceaObj.get_IvacCstrTc_ChmMwGam(Pc=self.Pc, MR=self.MR, eps=self.eps)
        return
    
    def calcEngineProperties(self):
        IspVac, Cstar, Tc, mw, gamma = self.ceaObj.get_IvacCstrTc_ChmMwGam(Pc=self.Pc, MR=self.MR, eps=self.eps)
        m_molar = mw/1000
        IspSea = self.ceaObj.estimate_Ambient_Isp(Pc=self.Pc, MR=self.MR, eps=self.eps, Pamb=1e5)
        Pc = self.Pc 


        Pe = Pc * nozzle.pressure_from_er(self.eps, gamma)
        # Empuxo no vacuo (N)
        thrustVac = nozzle.thrust(A_t = self.At,
                                  p_c = Pc,
                                  p_e = Pe,
                                  p_a = 0,
                                  gamma=gamma,
                                  er = self.eps)
        # Empuxo no nivel do mar (N)
        thrustSea = nozzle.thrust(A_t = self.At,
                                  p_c = Pc,
                                  p_e = Pe,
                                  p_a = 1e5,
                                  gamma=gamma,
                                  er = self.eps)
        # Fluxo de mass (kg/s)
        massFlow = nozzle.mass_flow(A_t = self.At,
                                     p_c = Pc,
                                     T_c = Tc,
                                     gamma = gamma,
                                     m_molar = m_molar
                                     )   

        print("Isp Vac (s): " + str(IspVac))
        print("Isp Sea (s): " + str(IspSea))
        print("Mass flow (kg/s): " + str(massFlow))
        print("Thrust Vac (kN): " + str(thrustVac/1000))
        print("Thrust Sea (kN): " + str(thrustSea/1000))

        self.IspVac = IspVac
        self.IspSea = IspSea
        self.massFlow = massFlow
        self.thrustVac = thrustVac
        self.thrustSea = thrustSea

    def estimateMass(self):
        ...



engine = EngineProp(fuelName="RP-1", 
                oxName="LOX",
                MR=2.36,
                Pc=9.7*1e6 ,
                At=0.042,
                eps=21.4)

engine.calcEngineProperties()


    


Isp Vac (s): 305.5677676625125
Isp Sea (s): (274.39544109463185, 'OverExpanded (Pe=1.39683e+07)')
Mass flow (kg/s): 294.4607412091103
Thrust Vac (kN): 905.7796996285897
Thrust Sea (kN): 815.8996996285897


# Estimando massa dos motores

In [22]:
def estimate_engine_mass(propellantType, thrustVac):
    """
    Estima a massa do motor
    
    :param proppelantType: Tipo de propelente - pode ser "Cryogenic-Cryogenic" ou "Cryogenic-Storable"
    :param thrustVac: Empuxo do motor no váculo em Newtons
    :return massTvc: Massa estimada do TVC em kg 
    """
    if propellantType == "Cryogenic-Cryogenic":
        engineMass = 7.54354 * (1e-3) * (thrustVac ** (8.85635 * (1e-1))) + 2.02881 * (1e1)
    elif propellantType == "Cryogenic-Storable":
        engineMass = 3.75407 * (1e3) * (thrustVac ** (7.05627 * (1e-2))) - 8.84790 * (1e3)
    else:
        raise Exception("Selecione um tipo de propelente válido!")
    return engineMass

def estimate_tvc_mass(thrustVac):
    """
    Estima a massa do Thrust Vector Control System (TVC)
    
    :param thrustVac: Empuxo do motor no váculo em Newtons
    :return massTvc: Massa estimada do TVC em kg 
    """
    massTvc = 0.1078 * (thrustVac/1e3) + 43.702
    return massTvc 


engineMass = estimate_engine_mass("Cryogenic-Storable",1817756)
massTvc = estimate_tvc_mass(1817756)
print(f"massa do motor: {engineMass}")
print(f"massa do TVC: {massTvc}")
print(f"massa total do sistema de propulsao: {engineMass + massTvc}")

massa do motor: 1531.9743933283971
massa do TVC: 239.65609680000003
massa total do sistema de propulsao: 1771.6304901283972


# Referências bibliográficas

- https://cearun.grc.nasa.gov/intro.html
- https://www.grc.nasa.gov/www/k-12/airplane/mflchk.html
- https://rocketcea.readthedocs.io/en/latest/functions.html
- http://www.braeunig.us/space/index.htm
- https://www.grc.nasa.gov/www/k-12/airplane/rktthsum.html
- https://www.grc.nasa.gov/www/k-12/rocket/nozzle.html
- http://mae-nas.eng.usu.edu/MAE_5420_Web/section5/section.5.3.pdf
- http://mae-nas.eng.usu.edu/MAE_5540_Web/propulsion_systems/MAE_5540_2022.html


# Motor Merlin-1D - Especificações
- https://www.wevolver.com/specs/merlin-engine-merlin-1d-falcon-9-falcon-heavy
- http://www.b14643.de/Spacerockets_2/United_States_1/Falcon-9/Merlin/index.htm

# Implementacoes
- https://proptools.readthedocs.io/en/latest/
- https://github.com/bluedack-space/OnlineRocketDesigner/blob/d8bb006de7a5c596b64008793a9bc778327d9143/rocketEngineHandler.py#L65

